In [1]:
#import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [ ]:
!unzip ml-1m.zip

In [ ]:
!unzip ml-100k.zip

In [4]:
#Prepare training set and test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [5]:
#Get number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print(nb_users)
print(nb_movies)

943
1682


In [6]:
#Convert data into an array with users in lines and movies in columns
def convert(data):
  #list of list is created and not 2d array
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:,1][data[:,0] == id_users]
    id_ratings = data[:,2][data[:,0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [7]:
#Convert data into Torch Tensors (multidimensional matrix)
training_set = torch.FloatTensor(training_set) 
test_set = torch.FloatTensor(test_set)

In [8]:
#Create architecture of Neural Network 
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 20) 
    self.fc2 = nn.Linear(20, 10) 
    self.fc3 = nn.Linear(10, 20) 
    self.fc4 = nn.Linear(20, nb_movies) 
    self.activation = nn.Sigmoid()
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x
sae = SAE()
criterion = nn.MSELoss()
optimiser = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay=0.5)

In [ ]:
#Train SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimiser.step()
    print('epoch '+str(epoch)+' loss: '+str(train_loss/s))  

In [10]:
#Test SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user])
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[(target == 0).unsqueeze(0)] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test_loss: '+str(test_loss/s)) 

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


test_loss: tensor(0.9492)
